# FIML

Aqui vou fazer o sistema polinomial sem considerar essa restrição do det(Gamma) = 1. Mas para as restrições vou usar o caso em que apenas algumas variáveis são usadas em cada caso.

Garantindo um sistema identificável, preciso de $g^2$ restrições. Então, vou fazer com que em cada equação o Y referente a ela seja coeficiente 1 (para ficar comparável com LIML), e selecionar aleatoriamente $g^2-g$ restrições para zerar (seja em B ou em Gamma).


$$ Y_{n x g} \Gamma_{g x g} = X_{n x k}B_{k x g} + U_{n x g} $$


Vamos considerar o problema do FIML primeiro para o caso homoskedastic. Assumimos que:  $U \sim N(0, \Sigma_{g x g} \otimes I_n)$. E vamos minimizar para o caso reduzido:

$$ Y = X B \Gamma^{-1} + U \Gamma^{-1}$$

A likelihood concentrada em $\Omega = B'^{-1} \Sigma B^{-1}$.

Ficaria um problema de $\min log |V'V|$. 


Sabemos que temos restrições, então ficamos com lagrangeano:

$$ log|V'V| - \lambda_1' (r_1 - R_1' vec(B)) - \lambda_2' (r_2 - R_2' vec(\Gamma)) $$


## (I)

Sistema polinomial final:

$$ vec(X'V) - ((V'V)\Gamma \otimes I_k)R_1\lambda_1 = 0 $$
$$ (I_g \otimes B'X')vec(V) - ((V'V) \otimes I_g)R_2 \lambda_2 = 0 $$
$$ R_1' vec(B) = r_1 $$
$$ R_2' vec(\Gamma) = r_2 $$




## (II)
Segunda forma que achei, considerando a questão de estar derivando em $\Gamma^{-1}$:

$$ (I_g \otimes B'X')vec(V) - ((V'V)\Gamma^{-1} \otimes \Gamma'^{-1})R_2 \lambda_2 = 0 $$
$$ vec(X'V) - ((V'V)\Gamma \otimes I_k)R_1\lambda_1 = 0 $$

e as restrições. O problema aqui é que dessa forma, derivei em $\Gamma^{-1}$, então estarei encontrando o resultado para ele. Enquanto, temos um $\Gamma$ nas equações. Então preciso considerar essa derivada.


# Método de Hausman


Também podemos usar a homotopia com o sistema de primeira ordem que Hausman (1975) encontrou.

## (III)

$$
\begin{bmatrix}
X' \\
(\Gamma')^{-1}B'X'
\end{bmatrix}
(Y\Gamma + XB)\Sigma ^{-1} = 0
$$

$$ \Sigma = (Y\Gamma - XB)'(Y\Gamma - XB)/n $$



## (IV)

Também temos a forma iterativa:

$$ \hat{\delta}_{k+1} = \left( \overline{W_k}' X \right)^{-1} \overline{W_k}' y $$



***
***
***

# Simulação: gerando os valores

Primeiro, especificamos a forma de gerar as matrizes $\Gamma$ e $B$. Note que ambas as matrizes estão dispostas de acordo com as restrições especificadas. 

Uma vez gerada as matrizes principais, precisamos também de uma função para encontrar as matrizes de restrição R1 e R2.

Colocamos uma matriz para checar qual a melhor solução entre as possíveis encontradas.

Em seguida, fazemos a geração aleatoria dos dados, considerando a seed 123.

In [1]:
function generate_B(k::Int, g::Int, r::Int; rng=Random.GLOBAL_RNG)
    M = zeros(Float64, k, g)
    
    # Passo 1: pelo menos um não-zero em cada linha
    for i in 1:k
        j = rand(rng, 1:g)
        val = 0
        while val == 0
            val = rand(rng, -10:10)   # inteiro aleatório diferente de zero
        end
        M[i, j] = val
    end

    # Passo 2: máximo de não-zeros permitido
    max_nonzeros = k*g - r
    current_nonzeros = count(!iszero, M)
    remaining = max_nonzeros - current_nonzeros

    # Passo 3: preencher o resto aleatoriamente
    positions = [(i,j) for i in 1:k for j in 1:g if M[i,j] == 0]
    shuffle!(positions)

    for (i,j) in Iterators.take(positions, remaining)
        val = 0
        while val == 0
            val = rand(rng, -10:10)
        end
        M[i,j] = val
    end
    
    return M
end


function generate_Gamma(g::Int, r::Int; rng=Random.GLOBAL_RNG)
    M = zeros(Float64, g, g)

    # Passo 1: fixa a diagonal em 1
    for i in 1:g
        M[i,i] = 1.0
    end

    # Passo 2: máximo de não-zeros permitido
    max_nonzeros = g^2 - r
    current_nonzeros = count(!iszero, M)
    remaining = max_nonzeros - current_nonzeros

    # Passo 3: preencher aleatoriamente fora da diagonal
    positions = [(i,j) for i in 1:g for j in 1:g if i != j && M[i,j] == 0]
    shuffle!(positions)

    for (i,j) in Iterators.take(positions, remaining)
        val = 0
        while val == 0 || val == 1
            val = rand(rng, -10:10)   # pode ser qualquer inteiro ≠ 0 e 1
        end
        M[i,j] = val
    end
    
    return M
end


function build_R(B::AbstractMatrix, Gamma::AbstractMatrix, g::Integer, k::Integer)
    R_1 = zeros(k*g,k*g)
    which_ones = findall(x -> x == 0, vec(B))

    for i in which_ones
        R_1[i,i] = 1
    end

    R_2 = zeros(g*g,g*g)
    which_ones = findall(x -> x == 0 || x == 1, vec(Gamma))

    for i in which_ones
        R_2[i,i] = 1
    end

    return R_1, R_2
end


function VpV(Y, X, Γ, B)
    V = Y - X * B * inv(Γ)
    return det(V' * V)
end



VpV (generic function with 1 method)

In [2]:
using LinearAlgebra, Random, Distributions
using Symbolics
using MultivariatePolynomials
using DynamicPolynomials
using HomotopyContinuation

Random.seed!(123)

n = 1000

k = 3
g = 2


# Restrições = g^2 - g
# r1 são restrições para B
# r2 são restrições para Gamma
r1 = 2
r2 = 0



X = rand(n, k)


Sigma = randn(g, g)
Sigma = Sigma' * Sigma




U = zeros(n,g)

d = MvNormal(zeros(g), Sigma)

for i in 1:n
    x = rand(d)
    U[i,:] = x
end


B = generate_B(k,g,r1)
Gamma = generate_Gamma(g, r2)


Y = X*B*inv(Gamma) + U*inv(Gamma)


R_1, R_2 = build_R(B, Gamma, g, k)

display(Sigma)


2×2 Matrix{Float64}:
  1.6795     -0.0515495
 -0.0515495   0.0425576

---
***
___

# Formas de resolver

## (I)


In [3]:
q1 = length(findall(x -> x != 0 , vec(B)))
q2 = length(findall(x -> x != 0 && x!= 1, vec(Gamma)))

@var b[1:q1] gam[1:q2]


Beta_est = zeros(Num,k,g)
Gamma_est = Matrix{Num}(I, g, g) 


it = 1

for i in 1:k
    for j in 1:g
        if(B[i,j] != 0)
            Beta_est[i,j] = b[it]
            it = it + 1
        end
    end
end



it = 1
for i in 1:g
    for j in 1:g
        if(Gamma[i,j] != 0 && Gamma[i,j] != 1)
            Gamma_est[i,j] = gam[it]
            it = it + 1
        end
    end
end




q1 = Int(sum(R_1))
q2 = Int(sum(R_2))

@var lam1[1:q1] lam2[1:q2]


lambda1 = zeros(Num,k*g,1)
lambda2 = zeros(Num,g*g,1)


it = 1

for i in 1:(k*g)
    if(R_1[i,i] != 0)
        lambda1[i] = lam1[it]
        it = it + 1
    end
end


it = 1

for i in 1:(g*g)
    if(R_2[i,i] != 0)
        lambda2[i] = lam2[it]
        it = it + 1
    end
end


#display(Beta_est)
#display(Gamma_est)

#display(lambda1)
#display(lambda2)



In [4]:
detG = det(Gamma_est)
V = (Symbolics.expand.(simplify.(Y*detG - X*Beta_est*detG*inv(Gamma_est))))

eqa1 = detG*vec(X'V) - kron((V'V)*Gamma_est,I(k))*lambda1 
eqa2 = detG*kron(I(g),Beta_est'*X')*vec(V) - kron((V'V),I(g))*lambda2 

eqf_1 = (simplify.(eqa1))
eqf_2 = (simplify.(eqa2))


F = vec( [eqf_1 ; eqf_2] ) 
vars = [ gam ; b ; lam1 ; lam2  ]
F_sys = System(F; variables=vars)

display("oi")

"oi"

In [5]:
result = solve(F_sys)

#result = solve(F_sys ; start_system = :total_degree)

rounded_solutions = [ real.(round.(sol; digits=4)) for sol in solutions(result)  ]

Tracking 933 paths... 100%|█████████████████████████████| Time: 0:00:28
                   # paths tracked: 933
   # non-singular solutions (real): 4 (4)
       # singular endpoints (real): 390 (0)
          # total solutions (real): 394 (4)


4-element Vector{Vector{Float64}}:
 [8.0333, -7.9738, -7.0333, -2.8557, 0.9912, 9.0203, 0.0, -0.0, -0.0, -0.0]
 [-0.0363, 0.1195, 0.0, 0.9543, 1.0799, -0.0, 0.8894, 0.8986, -0.0, -0.0]
 [-1.6346, -2.0613, 0.0, -1.4658, 0.6733, -1.2172, 0.0, 0.755, -0.0, 0.0]
 [-2.614, 0.0932, 0.4407, 0.601, -1.0382, 0.0, 0.8903, -0.0, 0.0, 0.0]

In [6]:
resultados_sol = []
for sol in solutions(result) 
    Aux = real.(round.(sol[1:6]; digits=2))

    Γ_eu = [  1      Aux[1];
            Aux[2]    1]
    
    B_eu = [Aux[3]   0;
            Aux[4]   Aux[5];
            0        Aux[6]]
    
    val = VpV(Y, X, Γ_eu, B_eu)
    push!(resultados_sol, (Γ = Γ_eu, Bs = B_eu, detVpV=val))
    display(val)
end


min_val, pos = findmin(r[:detVpV] for r in resultados_sol)

println("Mínimo = $min_val na posição $pos")

# agora pega a tupla correspondente
Resultado1 = resultados_sol[pos]

println("Par ótimo:")
println("Γ = ", Resultado1.Γ)
println("Bs = ", Resultado1.Bs)

16.24209967432221

21680.59614175916

82.96454385266071

3881.6569388842954

Mínimo = 16.24209967432221 na posição 1
Par ótimo:
Γ = [1.0 8.03; -7.97 1.0]
Bs = [-7.03 0.0; -2.86 0.99; 0.0 9.02]


# Formas de resolver

## (II)

In [28]:
q1 = length(findall(x -> x != 0 , vec(B)))
q2 = length(findall(x -> x != 0 && x!= 1, vec(Gamma)))

@var b[1:q1] gam[1:q2]


Beta_est = zeros(Num,k,g)
Gamma_est = Matrix{Num}(I, g, g) 


it = 1

for i in 1:k
    for j in 1:g
        if(B[i,j] != 0)
            Beta_est[i,j] = b[it]
            it = it + 1
        end
    end
end



it = 1
for i in 1:g
    for j in 1:g
        if(Gamma[i,j] != 0 && Gamma[i,j] != 1)
            Gamma_est[i,j] = gam[it]
            it = it + 1
        end
    end
end



q1 = Int(sum(R_1))
q2 = Int(sum(R_2))

@var lam1[1:q1] lam2[1:q2]


lambda1 = zeros(Num,k*g,1)
lambda2 = zeros(Num,g*g,1)


it = 1

for i in 1:(k*g)
    if(R_1[i,i] != 0)
        lambda1[i] = lam1[it]
        it = it + 1
    end
end


it = 1

for i in 1:(g*g)
    if(R_2[i,i] != 0)
        lambda2[i] = lam2[it]
        it = it + 1
    end
end


#display(Beta_est)
#display(Gamma_est)

#display(lambda1)
#display(lambda2)



$$ (I_g \otimes B'X')vec(V) - ((V'V)\Gamma^{-1} \otimes \Gamma'^{-1})R_2 \lambda_2 = 0 $$
$$ vec(X'V) - ((V'V)\Gamma \otimes I_k)R_1\lambda_1 = 0 $$

In [36]:
V = (Symbolics.expand.(simplify.(Y - X*Beta_est*(detG*inv(Gamma_est)))))


detG = det(Gamma_est)


eqa1 = (detG^3)*kron(I(g),Beta_est'*X')*vec(V) - kron( (V'V)*(detG*inv(Gamma_est)) ,(detG*inv(Gamma_est))') * lambda2 

eqa2 = detG*vec(X'*V) - kron((V'V)*(Gamma_est),I(k))*lambda1 

eqf_1 = (simplify.(eqa1))
eqf_2 = (simplify.(eqa2))


F = vec( [eqf_1 ; eqf_2] ) 
vars = [ gam ; b ; lam1 ; lam2  ]
F_sys = System(F; variables=vars)

display("oi")

"oi"

In [37]:
result = solve(F_sys)

#result = solve(F_sys ; start_system = :total_degree)

rounded_solutions = [ real.(round.(sol; digits=4)) for sol in solutions(result)  ]

Tracking 1251 paths... 100%|████████████████████████████| Time: 0:00:47
                   # paths tracked: 1251
   # non-singular solutions (real): 18 (10)
       # singular endpoints (real): 424 (0)
          # total solutions (real): 442 (10)


18-element Vector{Vector{Float64}}:
 [6.6904, -0.0498, -0.1461, 0.6822, 2.3175, 3.5412, 1.1843, 0.2138, -0.5572, -0.5572]
 [-2.614, 0.0932, 0.3544, 0.4833, -0.8349, 0.0, 1.1071, -0.0, 0.0, 0.0]
 [-2.6947, 0.1237, 0.4041, 0.4221, -0.5789, -0.3349, 1.1381, 0.0433, -0.1257, -0.1257]
 [-0.5881, 0.5668, 0.3539, 0.8278, 0.7558, -0.3837, 1.1193, 1.1699, -0.6493, -0.6493]
 [0.3633, -0.3033, -0.3306, 0.6069, 0.8507, 0.2759, 1.2312, 1.1979, -1.035, -1.035]
 [0.8375, -0.0736, 0.0911, 0.4881, 0.3281, 1.3697, 2.1775, -1.2691, -2.5741, -2.5741]
 [-4.7597, 0.07, 0.1685, 0.5507, -2.4354, 0.449, 1.252, -0.0491, 0.105, 0.105]
 [-0.4872, 0.0516, -0.3919, 0.9474, 1.7717, -1.6641, 0.9212, 0.3713, -0.5388, -0.5388]
 [6.5377, -0.051, -0.0519, 0.612, 0.9841, 5.2049, 1.2397, 0.9696, -0.8773, -0.8773]
 [0.8375, -0.0736, 0.0911, 0.4881, 0.3281, 1.3697, 2.1775, -1.2691, -2.5741, -2.5741]
 [0.3633, -0.3033, -0.3306, 0.6069, 0.8507, 0.2759, 1.2312, 1.1979, -1.035, -1.035]
 [-1.6346, -2.0613, 0.0, 0.6187, -0.2842, 0

In [41]:
Gamma

2×2 Matrix{Float64}:
  1.0  8.0
 -8.0  1.0

In [40]:
resultados_sol = []
for sol in solutions(result) 
    Aux = real.(round.(sol[1:6]; digits=2))

    Γ_eu = [  1      Aux[1];
            Aux[2]    1]
    
    B_eu = [Aux[3]   0;
            Aux[4]   Aux[5];
            0        Aux[6]]
    
    
#    Γ_eu = inv(Γ_eu)
    val = VpV(Y, X, Γ_eu, B_eu)
    push!(resultados_sol, (Γ = Γ_eu, Bs = B_eu, detVpV=val))
    display(val)
end


min_val, pos = findmin(r[:detVpV] for r in resultados_sol)

println("Mínimo = $min_val na posição $pos")

# agora pega a tupla correspondente
Resultado2 = resultados_sol[pos]

println("Par ótimo:")
println("Γ = ", Resultado2.Γ)
println("Bs = ", Resultado2.Bs)

30943.525145300846

6874.750423949811

7177.29924387398

18629.65388735653

20798.49562354775

...

27772.9739310348

21680.59614175916

3592.0094486341445

79343.02137533782

Mínimo = 3592.0094486341445 na posição 17
Par ótimo:
Γ = [1.0 0.16; -2.07 1.0]
Bs = [-0.03 0.0; -1.08 0.54; 0.0 0.52]


In [27]:
inv(Resultado2[1])

2×2 Matrix{Float64}:
 1.0   -8.03
 7.97   1.0

# Formas de resolver

## (III)

In [11]:
q1 = length(findall(x -> x != 0 , vec(B)))
q2 = length(findall(x -> x != 0 && x!= 1, vec(Gamma)))

@var bh[1:q1] gamh[1:q2]


Beta_esth = zeros(Num,k,g)
Gamma_esth = Matrix{Num}(I, g, g) 

it = 1

for i in 1:k
    for j in 1:g
        if(B[i,j] != 0)
            Beta_esth[i,j] = bh[it]
            it = it + 1
        end
    end
end


it = 1
for i in 1:g
    for j in 1:g
        if(Gamma[i,j] != 0 && Gamma[i,j] != 1)
            Gamma_esth[i,j] = gamh[it]
            it = it + 1
        end
    end
end

#display(Beta_esth)
#display(Gamma_esth)


In [12]:
detGh = det(Gamma_esth')
Uh = Y*Gamma_esth - X*Beta_esth
Auxeq = (detGh*inv(Gamma_esth'))*Beta_esth'


eqa1 = vec(X'Uh) 

eqa2 = vec(Auxeq*X'Uh)

eqf_1 = (simplify.(eqa1))
eqf_2 = (simplify.(eqa2))


F = vec( [eqf_1 ; eqf_2] ) 

vars = [ gamh ; bh]

F_sys = System(F; variables=vars)

display("oi")


"oi"

In [13]:
result = solve(F_sys)
rounded_solutions = [ real.(round.(sol; digits=4)) for sol in solutions(result)  ]


Tracking 400 paths... 100%|█████████████████████████████| Time: 0:01:48
                   # paths tracked: 400
   # non-singular solutions (real): 1 (1)
       # singular endpoints (real): 0 (0)
          # total solutions (real): 1 (1)


1-element Vector{Vector{Float64}}:
 [8.0333, -7.9738, -7.0333, -2.8557, 0.9912, 9.0203]

In [14]:
resultados_sol = []
for sol in solutions(result) 
    Aux = real.(round.(sol[1:6]; digits=2))

    Γ_eu = [  1      Aux[1];
            Aux[2]    1]
    
    B_eu = [Aux[3]   0;
            Aux[4]   Aux[5];
            0        Aux[6]]
    
    val = VpV(Y, X, Γ_eu, B_eu)
    push!(resultados_sol, (Γ = Γ_eu, Bs = B_eu, detVpV=val))
#    display(val)
end


min_val, pos = findmin(r[:detVpV] for r in resultados_sol)

println("Mínimo = $min_val na posição $pos")

# agora pega a tupla correspondente
Resultado3 = resultados_sol[pos]

println("Par ótimo:")
println("Γ = ", Resultado3.Γ)
println("Bs = ", Resultado3.Bs)

Mínimo = 16.24209967432221 na posição 1
Par ótimo:
Γ = [1.0 8.03; -7.97 1.0]
Bs = [-7.03 0.0; -2.86 0.99; 0.0 9.02]


# Formas de resolver

## (IV)

In [15]:
using LinearAlgebra, Statistics

# util: bloco diagonal
function blkdiag(mats::AbstractMatrix...)
    r = sum(size(m,1) for m in mats)
    c = sum(size(m,2) for m in mats)
    M = zeros(eltype(mats[1]), r, c)
    ri = 1; ci = 1
    for A in mats
        rr, cc = size(A)
        M[ri:ri+rr-1, ci:ci+cc-1] .= A
        ri += rr; ci += cc
    end
    return M
end

function hausman_iter_sem(Y::AbstractMatrix, Z::AbstractMatrix;
        endog_sets::Vector{Vector{Int}},
        exog_sets::Vector{Vector{Int}},
        inst_sets::Union{Nothing,Vector{Vector{Int}}}=nothing,
        maxiter::Int=200, tol::Float64=1e-8, verbose::Bool=true)

    T, M = size(Y)
    K = size(Z,2)
    @assert length(endog_sets)==M && length(exog_sets)==M

    # instrumentos por equação
    if inst_sets === nothing
        inst_sets = [collect(1:K) for _ in 1:M]   # todos Z como instrumentos
    end

    # constrói blocos de regressoras (X_i) e instrumentos (Z_i) por equação
    function build_blocks()
        Xblocks = Vector{Matrix{Float64}}(undef, M)
        Zblocks = Vector{Matrix{Float64}}(undef, M)
        yblocks = Vector{Vector{Float64}}(undef, M)
        for i in 1:M
            Xi = hcat(Y[:, endog_sets[i]], Z[:, exog_sets[i]])
            Zi = Z[:, inst_sets[i]]
            Xblocks[i] = Array(Xi)
            Zblocks[i] = Array(Zi)
            yblocks[i] = Array(Y[:, i])
        end
        return Xblocks, Zblocks, yblocks
    end

    Xb, Zb, yb = build_blocks()
    # empilha em bloco diagonal
    X = blkdiag(Xb...)
    Zsys = blkdiag(Zb...)
    y = vcat(yb...)

    # projeção por 2SLS por equação (inicial consistente)
    function twoSLS_per_eq(yb, Xb, Zb)
        βhat = Vector{Vector{Float64}}(undef, M)
        for i in 1:M
            Zi, Xi, yi = Zb[i], Xb[i], yb[i]
            Pz = Zi * inv(Zi'Zi) * Zi'
            βhat[i] = inv(Xi'Pz*Xi) * (Xi'Pz*yi)
        end
        return βhat
    end
    βblocks = twoSLS_per_eq(yb, Xb, Zb)
    β = vcat(βblocks...)

    # funções auxiliares
    # residuals por equação, dados β
    function residuals(β::Vector{Float64})
        res = zeros(T, M)
        idx = 1
        for i in 1:M
            pi = size(Xb[i],2)
            b_i = β[idx:idx+pi-1]
            res[:, i] = yb[i] .- Xb[i]*b_i
            idx += pi
        end
        return res
    end

    # log-likelihood concentrada (up to constants): - (T/2) log det(Σ)
    llhist = Float64[]
    function conc_loglik(Σ::Matrix{Float64})
        # ignorando termos constantes e |det(B)| (absorvido pela parametrização);
        # útil só como monitor monotônico aproximado
        return -0.5*T*log(det(Σ))
    end

    converged = false
    for it in 1:maxiter
        # (1) estima Σ a partir dos resíduos estruturais
        U = residuals(β)
        Σ = (U'U)/T

        # (2) peso GLS de sistema: Ω^{-1} = Σ^{-1} ⊗ I_T
        Σinv = inv(Symmetric(Σ))
        Winv = kron(Σinv, I(T))

        # (3) passo FIIV/3SLS (sistema IV ponderado)
        #     β_{new} = (X' W Z (Z' W Z)^{-1} Z' W X)^{-1} X' W Z (Z' W Z)^{-1} Z' W y
        ZWZ = Zsys' * Winv * Zsys
        PzW = Zsys * inv(Symmetric(ZWZ)) * Zsys'   # "projeção" ponderada
        XWPZX = X' * Winv * PzW * X
        XWPZy = X' * Winv * PzW * y
        β_new = inv(Symmetric(XWPZX)) * XWPZy

        # (4) checa convergência
        diff = maximum(abs.(β_new .- β))
        push!(llhist, conc_loglik(Σ))
        if verbose
            @info "iter $it |Δβ|∞ = $(round(diff, sigdigits=4))"
        end
        if diff < tol
            β = β_new
            converged = true
            break
        end
        β = β_new
    end

    # arruma saída por equação
    βblocks = Vector{Vector{Float64}}(undef, M)
    idx = 1
    for i in 1:M
        pi = size(Xb[i],2)
        βblocks[i] = β[idx:idx+pi-1]
        idx += pi
    end

    return (β = β, β_blocks = βblocks, Σ = (residuals(β)'*residuals(β))/T,
            ll = llhist, converged = converged)
end


hausman_iter_sem (generic function with 1 method)

In [16]:
res = hausman_iter_sem(Y, X;
    endog_sets = [[2], [1]],
    exog_sets  = [[1,2], [2,3]],
    inst_sets  = nothing,   # usa todos os Z como instrumentos
    tol = 1e-8,
    maxiter = 1000,
    verbose = true
)




[ Info: iter 1 |Δβ|∞ = 1.192e-12


(β = [7.973843762901175, -7.033288971304439, -2.8557359510057534, -8.033271935495263, 0.9912225739161161, 9.020312240602623], β_blocks = [[7.973843762901175, -7.033288971304439, -2.8557359510057534], [-8.033271935495263, 0.9912225739161161, 9.020312240602623]], Σ = [1.689335114656949 -0.05269254717142278; -0.05269254717142278 0.04233277564977142], ll = [1338.728443687886], converged = true)

In [17]:
Γ_eu = [  1      res[1][1];
        res[1][4]    1]

B_eu = [res[1][2]   0;
        res[1][3]   res[1][5];
        0           res[1][6]]

val = VpV(Y, X, Γ_eu, B_eu)
Resultado4 = (Γ = Γ_eu, Bs = B_eu, detVpV=val)

(Γ = [1.0 7.973843762901175; -8.033271935495263 1.0], Bs = [-7.033288971304439 0.0; -2.8557359510057534 0.9912225739161161; 0.0 9.020312240602623], detVpV = 16.84461185798735)

# Comparar os resultados de cada método

Temos 4 soluções, e ver como cada uma recuperou o valor verdadeiro.


In [18]:
display(B)
display(Gamma)

3×2 Matrix{Float64}:
 -7.0  0.0
 -3.0  1.0
  0.0  9.0

2×2 Matrix{Float64}:
  1.0  8.0
 -8.0  1.0

In [21]:
display(Resultado1[1])
display(Resultado2[1])
display(Resultado3[1])
display(Resultado4[1])

2×2 Matrix{Float64}:
  1.0   8.03
 -7.97  1.0

2×2 Matrix{Float64}:
 1.0   -0.84
 0.07   1.0

2×2 Matrix{Float64}:
  1.0   8.03
 -7.97  1.0

2×2 Matrix{Float64}:
  1.0      7.97384
 -8.03327  1.0

# Function to check minimum of V'V



In [ ]:
function VpV(Y, X, Γ, B)
    V = Y - X * B * inv(Γ)
    return det(V' * V)
end

In [ ]:
rounded_solutions = [ real.(round.(sol[1:6]; digits=2)) for sol in solutions(result)  ]

In [ ]:
length(solutions(result))

In [ ]:
for sol in solutions(result) 
    resultados_sol = []
    Aux = real.(round.(sol[1:6]; digits=2))

    Γ_eu = [  1      Aux[1];
            Aux[2]    1]
    
    B_eu = [Aux[3]   0;
            Aux[4]   Aux[5];
            0        Aux[6]]
    
    val = VpV(Y, X, Γ_eu, B_eu)
    push!(resultados_sol, (Γ = Γ_eu, Bs = B_eu, detVpV=val))
    display(val)
end

In [ ]:
Aux = rounded_solutions[1]

In [ ]:
Aux = rounded_solutions[1]
resultados_sol = []
for i1 in [0; -0.1; 0.1], i2 in [0; -0.1; 0.1], i3 in [0; -0.1; 0.1], i4 in [0; -0.1; 0.1], i5 in [0; -0.1; 0.1], i6 in [0; -0.1; 0.1]


    Γ_eu = [  1      Aux[1] + i1;
            Aux[2]+i2    1]
    
    B_eu = [Aux[3]+i3   0;
            Aux[4]+i4   Aux[5]+i5;
            0        Aux[6]+i6]
    
    val = VpV(Y, X, Γ_eu, B_eu)
    push!(resultados_sol, (Γ = Γ_eu, Bs = B_eu, detVpV=val))
end

In [ ]:
resultados_sol

In [ ]:
min_val, pos = findmin(r[:detVpV] for r in resultados_sol)

println("Mínimo = $min_val na posição $pos")

# agora pega a tupla correspondente
min_tuple = resultados_sol[pos]

println("Par ótimo:")
println("Γ = ", min_tuple.Γ)
println("Bs = ", min_tuple.Bs)

In [ ]:
Γ_1 = [1 8.03;
      -7.97 1]
B_1 = [-7.03 0;
       -2.86 0.99;
        0   9.02]

In [ ]:
U1 = Y*Γ_1 - X*B_1

Sigma1 = (U1'U1/n)

In [ ]:
Sigma

# Fazer um super grid

Essa seção ainda não terminei

In [ ]:
function det_grid(Y, X)
    results = []

    for γ1 in -10:1:10, β1 in -10:1:10, β2 in -10:1:10, β3 in -10:1:10
        Γ  = [1 γ1;
              0 1]
        Bs = [β1 0;
              β2 β3]
        
        val = VpV(Y, X, Γ, B)
        push!(results, (Γ = Γ, Bs = Bs, detVpV=val))
    end

    return results
end

In [ ]:
resultado = det_grid(Y,X)

In [ ]:
min_val, pos = findmin(r[:detVpV] for r in resultado)

println("Mínimo = $min_val na posição $pos")

# agora pega a tupla correspondente
min_tuple = resultado[pos]

println("Par ótimo:")
println("Γ = ", min_tuple.Γ)
println("Bs = ", min_tuple.Bs)

# Teste de overidentificação


Encontrar valor da ll do FIML.

Encontrar valor da ll do unrestricted reduced form (URF) ~ vindo do OLS

2(l_1 - l_2) ~ Chi Quadrada

In [ ]:
PI = inv(X'X)X'Y

In [ ]:
size(X)

In [ ]:
ll1 = -n * log(   det(   (Y - X*PI)' * (Y - X*PI)/n))/2


In [ ]:
display(ll1)
display(ll2)

In [ ]:
ll1 = -n * log(   det(   (Y - X*PI)' * (Y - X*PI)/n))/2


ll2 = -n * log( det((Y - X*B_1*inv(Γ_1))' * (Y - X*B_1*inv(Γ_1))/n))/2
            
2*(ll1 - ll2)